In [58]:
from pytorch_tabnet.tab_model import TabNetClassifier

import os
import torch
import pandas as pd
import numpy as np
 
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [59]:
#Usamos el GPU si esta disponible, mps porque es apple silicon.
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"using {device}")

#también hay que mover los datos a la gpu.

using mps


Cargar los datos y estructurarlos para entrenar el modelo.

Voy a usar los datos que usó Diego.

In [60]:
folder = "Data"
match_file = "estadisticas_partidos.csv"

path_matches = os.path.join(folder, match_file)
matches = pd.read_csv(path_matches)

In [61]:
print(matches.columns.values)
print(len(matches.columns.values))

['team_home' 'team_away' 'score_home' 'score_away' 'expected_goals_home'
 'expected_goals_away' 'posession_home' 'posession_away'
 'total_shots_home' 'total_shots_away' 'shots_on_target_home'
 'shots_on_target_away' 'shots_off_target_home' 'shots_off_target_away'
 'blocked_shots_home' 'blocked_shots_away' 'corner_kicks_home'
 'corner_kicks_away' 'offsides_home' 'offsides_away' 'fouls_home'
 'fouls_away' 'yellow_cards_home' 'yellow_cards_away' 'red_cards_home'
 'red_cards_away' 'free_kicks_home' 'free_kicks_away' 'throw_ins_home'
 'throw_ins_away' 'goal_kicks_home' 'goal_kicks_away'
 'shots_inside_box_home' 'shots_inside_box_away' 'shots_outside_box_home'
 'shots_outside_box_away' 'goalkeeper_saves_home' 'goalkeeper_saves_away'
 'passes_home' 'passes_away' 'accurate_passes_home' 'accurate_passes_away'
 'long_balls_home' 'long_balls_away' 'crosses_home' 'crosses_away'
 'dribbles_home' 'dribbles_away' 'possesion_lost_home'
 'possesion_lost_away' 'duels_won_home' 'duels_won_away'
 'aerials

In [62]:
matches.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 60 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   team_home               240 non-null    object 
 1   team_away               240 non-null    object 
 2   score_home              240 non-null    int64  
 3   score_away              240 non-null    int64  
 4   expected_goals_home     240 non-null    float64
 5   expected_goals_away     240 non-null    float64
 6   posession_home          240 non-null    int64  
 7   posession_away          240 non-null    int64  
 8   total_shots_home        240 non-null    int64  
 9   total_shots_away        240 non-null    int64  
 10  shots_on_target_home    240 non-null    int64  
 11  shots_on_target_away    240 non-null    int64  
 12  shots_off_target_home   240 non-null    int64  
 13  shots_off_target_away   240 non-null    int64  
 14  blocked_shots_home      240 non-null    in

In [63]:
#Aqui agrega la columna de resultados

results = []

for index in matches.index:

    match = matches.loc[index]
    if match["score_home"] > match["score_away"]:
        results.append(1)
    elif match["score_home"] < match["score_away"]:
        results.append(2)
    else:
        results.append(0)

matches["result"] = results


In [64]:
matches.head()

,team_home,team_away,score_home,score_away,expected_goals_home,expected_goals_away,posession_home,posession_away,total_shots_home,total_shots_away,...,duels_won_away,aerials_won_home,aerials_won_away,tackles_home,tackles_away,interceptions_home,interceptions_away,clearences_home,clearences_away,result
0,Curicó Unido,Colo Colo,0,1,0.07,2.31,32,68,5,20,...,33,4,11,11,8,5,8,23,8,2
1,Huachipato,Audax Italiano,2,0,1.34,0.45,64,36,12,13,...,35,13,9,12,12,7,4,19,28,1
2,Unión Española,Cobresal,1,0,1.09,0.54,65,35,26,9,...,66,18,14,18,19,3,11,24,24,1
3,Deportes Copiapó,Everton de Viña del Mar,2,0,1.21,0.59,36,64,10,12,...,35,11,9,13,9,7,2,28,13,1
4,Deportes Magallanes,Coquimbo Unido,2,3,1.62,1.82,57,43,13,15,...,32,9,11,10,6,5,1,16,20,2


In [65]:
matches.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 61 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   team_home               240 non-null    object 
 1   team_away               240 non-null    object 
 2   score_home              240 non-null    int64  
 3   score_away              240 non-null    int64  
 4   expected_goals_home     240 non-null    float64
 5   expected_goals_away     240 non-null    float64
 6   posession_home          240 non-null    int64  
 7   posession_away          240 non-null    int64  
 8   total_shots_home        240 non-null    int64  
 9   total_shots_away        240 non-null    int64  
 10  shots_on_target_home    240 non-null    int64  
 11  shots_on_target_away    240 non-null    int64  
 12  shots_off_target_home   240 non-null    int64  
 13  shots_off_target_away   240 non-null    int64  
 14  blocked_shots_home      240 non-null    in

In [66]:
#Cantidad de partidos por resultado (local, empate, visita)

for i in range(0,3):
    cantidad = matches[matches["result"]==i].shape[0]
    print(f"Resultado {i}, hay: {cantidad} partidos")
num = 97/(59+97+84)
print(num)

Resultado 0, hay: 59 partidos
Resultado 1, hay: 97 partidos
Resultado 2, hay: 84 partidos
0.4041666666666667


In [67]:
result = matches["result"]
matches = matches.drop(columns = ["team_home", "team_away", "result", "score_home", "score_away"])
matches.head()

columnas_categoricas = matches.select_dtypes(include=['object']).columns.tolist()

print("Columnas categóricas:")
print(columnas_categoricas)

Columnas categóricas:
[]


In [68]:
# Convertir columnas categoricas a numericas

#col = LabelEncoder().fit(matches["columna_tabla"])
#data["columna_tabla"] = col.transform(matches["columna_tabla"])

In [95]:
X = matches.to_numpy()
scaler = StandardScaler()
X_escalados = scaler.fit_transform(X)
y = result

In [96]:
#X_train, X_test, y_train, y_test = train_test_split(X_escalados, y, test_size=0.3, random_state = 1, stratify=y)

In [104]:
kfold = KFold(n_splits=5)

y_true_all = []
y_pred_all = []

for train_index, test_index in kfold.split(X_escalados):
    X_train, X_test = X_escalados[train_index], X_escalados[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = TabNetClassifier(
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=1e-3),
        scheduler_params={"step_size":50, # how to use learning rate scheduler
                            "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax', # "sparsemax" "entmax"
        )
        
    clf.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)], 
        eval_name=['train', 'valid'], 
        eval_metric=['accuracy'], 
        max_epochs=1000 , patience=100, 
        batch_size=28, drop_last=False)

    pred = clf.predict(X_test)
    # Almacenar las predicciones y las etiquetas verdaderas de este fold
    y_true_all.extend(y_test)
    y_pred_all.extend(pred)


/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.15431 | train_accuracy: 0.34896 | valid_accuracy: 0.4375  |  0:00:00s
epoch 1  | loss: 1.18652 | train_accuracy: 0.36458 | valid_accuracy: 0.39583 |  0:00:00s
epoch 2  | loss: 1.15968 | train_accuracy: 0.42708 | valid_accuracy: 0.33333 |  0:00:00s
epoch 3  | loss: 1.20191 | train_accuracy: 0.40625 | valid_accuracy: 0.39583 |  0:00:00s
epoch 4  | loss: 1.15111 | train_accuracy: 0.41667 | valid_accuracy: 0.4375  |  0:00:00s
epoch 5  | loss: 1.13387 | train_accuracy: 0.4375  | valid_accuracy: 0.45833 |  0:00:00s
epoch 6  | loss: 1.16715 | train_accuracy: 0.43229 | valid_accuracy: 0.45833 |  0:00:00s
epoch 7  | loss: 1.1886  | train_accuracy: 0.45312 | valid_accuracy: 0.4375  |  0:00:00s
epoch 8  | loss: 1.10109 | train_accuracy: 0.46354 | valid_accuracy: 0.45833 |  0:00:00s
epoch 9  | loss: 1.11418 | train_accuracy: 0.47396 | valid_accuracy: 0.47917 |  0:00:00s
epoch 10 | loss: 1.07827 | train_accuracy: 0.5     | valid_accuracy: 0.47917 |  0:00:00s
epoch 11 | loss: 1.12

/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 1  | loss: 1.15766 | train_accuracy: 0.38542 | valid_accuracy: 0.41667 |  0:00:00s
epoch 2  | loss: 1.11117 | train_accuracy: 0.41667 | valid_accuracy: 0.39583 |  0:00:00s
epoch 3  | loss: 1.11695 | train_accuracy: 0.43229 | valid_accuracy: 0.4375  |  0:00:00s
epoch 4  | loss: 1.10333 | train_accuracy: 0.46354 | valid_accuracy: 0.45833 |  0:00:00s
epoch 5  | loss: 1.10079 | train_accuracy: 0.46354 | valid_accuracy: 0.4375  |  0:00:00s
epoch 6  | loss: 1.12493 | train_accuracy: 0.47917 | valid_accuracy: 0.41667 |  0:00:00s
epoch 7  | loss: 1.12894 | train_accuracy: 0.45833 | valid_accuracy: 0.39583 |  0:00:00s
epoch 8  | loss: 1.0539  | train_accuracy: 0.44271 | valid_accuracy: 0.35417 |  0:00:00s
epoch 9  | loss: 1.13036 | train_accuracy: 0.44792 | valid_accuracy: 0.375   |  0:00:00s
epoch 10 | loss: 1.08577 | train_accuracy: 0.44271 | valid_accuracy: 0.39583 |  0:00:00s
epoch 11 | loss: 1.06219 | train_accuracy: 0.44792 | valid_accuracy: 0.39583 |  0:00:00s
epoch 12 | loss: 1.07

/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 2  | loss: 1.13193 | train_accuracy: 0.41146 | valid_accuracy: 0.35417 |  0:00:00s
epoch 3  | loss: 1.1455  | train_accuracy: 0.42188 | valid_accuracy: 0.3125  |  0:00:00s
epoch 4  | loss: 1.10844 | train_accuracy: 0.43229 | valid_accuracy: 0.3125  |  0:00:00s
epoch 5  | loss: 1.16993 | train_accuracy: 0.43229 | valid_accuracy: 0.29167 |  0:00:00s
epoch 6  | loss: 1.1317  | train_accuracy: 0.44792 | valid_accuracy: 0.33333 |  0:00:00s
epoch 7  | loss: 1.13085 | train_accuracy: 0.44792 | valid_accuracy: 0.33333 |  0:00:00s
epoch 8  | loss: 1.08409 | train_accuracy: 0.45833 | valid_accuracy: 0.35417 |  0:00:00s
epoch 9  | loss: 1.14956 | train_accuracy: 0.47396 | valid_accuracy: 0.39583 |  0:00:00s
epoch 10 | loss: 1.08056 | train_accuracy: 0.5     | valid_accuracy: 0.375   |  0:00:00s
epoch 11 | loss: 1.15096 | train_accuracy: 0.51042 | valid_accuracy: 0.375   |  0:00:00s
epoch 12 | loss: 1.10588 | train_accuracy: 0.48958 | valid_accuracy: 0.35417 |  0:00:00s
epoch 13 | loss: 1.08

/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.1589  | train_accuracy: 0.39062 | valid_accuracy: 0.22917 |  0:00:00s
epoch 1  | loss: 1.15907 | train_accuracy: 0.39062 | valid_accuracy: 0.3125  |  0:00:00s
epoch 2  | loss: 1.13093 | train_accuracy: 0.4375  | valid_accuracy: 0.33333 |  0:00:00s
epoch 3  | loss: 1.13752 | train_accuracy: 0.4375  | valid_accuracy: 0.35417 |  0:00:00s
epoch 4  | loss: 1.11492 | train_accuracy: 0.44271 | valid_accuracy: 0.39583 |  0:00:00s
epoch 5  | loss: 1.22809 | train_accuracy: 0.43229 | valid_accuracy: 0.39583 |  0:00:00s
epoch 6  | loss: 1.12207 | train_accuracy: 0.44271 | valid_accuracy: 0.41667 |  0:00:00s
epoch 7  | loss: 1.13545 | train_accuracy: 0.44792 | valid_accuracy: 0.47917 |  0:00:00s
epoch 8  | loss: 1.12034 | train_accuracy: 0.44271 | valid_accuracy: 0.52083 |  0:00:00s
epoch 9  | loss: 1.12698 | train_accuracy: 0.44792 | valid_accuracy: 0.52083 |  0:00:00s
epoch 10 | loss: 1.12247 | train_accuracy: 0.46875 | valid_accuracy: 0.52083 |  0:00:00s
epoch 11 | loss: 1.11

/Users/gustavofernandez/Desktop/pytorch-test/env/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [102]:
print("classification report")
print(classification_report(y_true_all, y_pred_all))


classification report
              precision    recall  f1-score   support

           0       0.39      0.22      0.28        59
           1       0.55      0.72      0.62        97
           2       0.57      0.54      0.55        84

    accuracy                           0.53       240
   macro avg       0.50      0.49      0.49       240
weighted avg       0.52      0.53      0.51       240



In [ ]:
cnf_matrix = confusion_matrix(y_true_all, y_pred_all)
np.set_printoptions(precision=2)
# Plot normalized confusion matrix
plt.figure(figsize=(5,5))
plot_confusion_matrix(cnf_matrix, classes=classes, normalize=True, title='Normalized confusion matrix')

plt.show()